In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setAppName("PartitioningExample").setMaster("local[*]")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/07 23:38:51 WARN Utils: Your hostname, kingstein, resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp3s0)
25/11/07 23:38:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/07 23:38:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = list(range(1,21))

In [5]:
rdd = sc.parallelize(data,4)
print("INitial numbers of Partitions: ", rdd.getNumPartitions())

INitial numbers of Partitions:  4


In [6]:
def inspect_partitions(index,iterator):
    yield f"Partition {index}:{list(iterator)}"

In [7]:
print("\n Original Partition Layout:")
for part in rdd.mapPartitionsWithIndex(inspect_partitions).collect():
    print(part)


 Original Partition Layout:


Partition 0:[1, 2, 3, 4, 5]
Partition 1:[6, 7, 8, 9, 10]
Partition 2:[11, 12, 13, 14, 15]
Partition 3:[16, 17, 18, 19, 20]


In [8]:
rdd_repartitioned = rdd.repartition(6)
print("\nAfter repartition(6):")
print("Partitions:", rdd_repartitioned.getNumPartitions())
for part in rdd_repartitioned.mapPartitionsWithIndex(inspect_partitions).collect():
    print(part)


After repartition(6):
Partitions: 6
Partition 0:[16, 17, 18, 19, 20]
Partition 1:[]
Partition 2:[]
Partition 3:[1, 2, 3, 4, 5]
Partition 4:[6, 7, 8, 9, 10]
Partition 5:[11, 12, 13, 14, 15]


In [11]:
rdd_coalesced = rdd.coalesce(2)
print("\n After coalesce(2)")
print("Partitions:",rdd_coalesced.getNumPartitions())
for part in rdd_coalesced.mapPartitionsWithIndex(inspect_partitions).collect():
    print(part)


 After coalesce(2)
Partitions: 2
Partition 0:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Partition 1:[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
